In [1]:
import numpy as np
from time import process_time
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.linear_model import Ridge
import imblearn

import warnings
warnings.filterwarnings("ignore")

N = 4

pd = '' #path with data

In [2]:
data = loadmat(pd+'dataAnswer_regular_from_0_to_3pion4.mat')
answers_reg_ = np.transpose(np.array(data['answers']))
data = loadmat(pd+'dataChoi3_sn_regular_from_0_to_3pion4.mat')
chois_reg_ = np.array(data['choi'])

data = loadmat(pd+'dataAnswer_regular_otherY_from_0_to_3pion4.mat')
answers_Y = np.transpose(np.array(data['answers']))
data = loadmat(pd+'dataChoi3_sn_regular_otherY_from_0_to_3pion4.mat')
chois_Y = np.array(data['choi'])

chois_short = np.concatenate((chois_reg_, chois_Y), axis=0)
answers_reg = np.concatenate((answers_reg_, answers_Y), axis=0)


data = loadmat(pd+'dataChoi3_n_regular_from_0_to_3pion4.mat')
chois_reg_ = np.array(data['choi'])
data = loadmat(pd+'dataChoi3_n_regular_otherY_from_0_to_3pion4.mat')
chois_Y = np.array(data['choi'])

chois_long = np.concatenate((chois_reg_, chois_Y), axis=0)

data = loadmat(pd+'dataAnswer_regular_3pion8.mat')
answers_other1 = np.transpose(np.array(data['answers']))
data = loadmat(pd+'dataChoi3_sn_regular_3pion8.mat')
chois_other1 = np.array(data['choi'])

data = loadmat(pd+'dataAnswer_regular_otherY_3pion8.mat')
answers_other2 = np.transpose(np.array(data['answers']))
data = loadmat(pd+'dataChoi3_sn_regular_otherY_3pion8.mat')
chois_other2 = np.array(data['choi'])

chois_short_other = np.concatenate((chois_other1, chois_other2), axis=0)
answers_other = np.concatenate((answers_other1, answers_other2), axis=0)


data = loadmat(pd+'dataChoi3_n_regular_3pion8.mat')
chois_other1 = np.array(data['choi'])
data = loadmat(pd+'dataChoi3_n_regular_otherY_3pion8.mat')
chois_other2 = np.array(data['choi'])

chois_long_other = np.concatenate((chois_other1, chois_other2), axis=0)

In [3]:
X = chois_long
y = answers_reg

In [4]:
classifiers = KNeighborsClassifier(3)
metrics_name = ["euclidean", "manhattan", "chebyshev", "minkowski"]


In [5]:
X = chois_long
y = answers_reg

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.1, random_state=777)

X_other = chois_long_other
y_other = answers_other

for name in metrics_name:
    clf = KNeighborsClassifier(3, metric=name)
    clf.fit(X_train, y_train)
    score0 = clf.score(X_train, y_train)
    score = clf.score(X_test, y_test)
    score_other = clf.score(X_other, y_other)
    y_t = clf.predict(X_train)
    y_p = clf.predict(X_other)
       
    f1_t = metrics.f1_score(y_train, y_t)
    f1_p = metrics.f1_score(y_other, y_p)
    auc_t = metrics.roc_auc_score(y_train, y_t)
    auc_p = metrics.roc_auc_score(y_other, y_p)
    
    print((name+" {:.5f};   {:.5f};   {:.5f};   {:.5f};   {:.5f};   {:.5f};   {:.5f}").format(score0, score, score_other, f1_t, f1_p, auc_t, auc_p))    

euclidean 0.99556;   0.98783;   0.97580;   0.99666;   0.98300;   0.99442;   0.97277
manhattan 0.99604;   0.98867;   0.97650;   0.99702;   0.98347;   0.99504;   0.97463
chebyshev 0.99454;   0.98633;   0.97350;   0.99589;   0.98142;   0.99331;   0.96886
minkowski 0.99556;   0.98783;   0.97580;   0.99666;   0.98300;   0.99442;   0.97277


In [6]:

from sklearn.model_selection import GridSearchCV
estimator_KNN = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': (1,10, 1),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev')}
                   
grid_search_KNN = GridSearchCV(
    estimator=estimator_KNN,
    param_grid=parameters_KNN,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 2
)

In [7]:
KNN_1=grid_search_KNN.fit(X_train, y_train)
y_pred_KNN1 =KNN_1.predict(X_other)

#Parameter setting that gave the best results on the hold out data.
print(grid_search_KNN.best_params_ ) 
#Mean cross-validated score of the best_estimator
print('Best Score - KNN:', grid_search_KNN.best_score_ )

{'leaf_size': 1, 'metric': 'minkowski', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
Best Score - KNN: 0.9841851851851852


In [8]:
scor_p = metrics.accuracy_score(y_other, y_pred_KNN1)
f1_p = metrics.f1_score(y_other, y_pred_KNN1)
auc_p = metrics.roc_auc_score(y_other, y_pred_KNN1)
print([scor_p, f1_p, auc_p])

[0.9744, 0.9819971870604781, 0.972003694327154]
